### Práctica de Formatos de datos: XML

El objetivo de esta práctica es crear una aplicación que gestionará información sobre las bibliotecas públicas y bibliobuses situados en la ciudad de Madrid, de titularidad nacional, comunitaria o local. 

Para ello se va a usar el catálogo "Bibliotecas y bibliobuses en la ciudad de Madrid" del portal de datos abiertos del Ayuntamiento de Madrid. Este catalogo ofrece datos de dirección, horario, servicios y coordenadas para su georeferenciación. La información se encuentra en:
https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ed35401429b83410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

Y el conjunto de datos XML que hay que procesar está situado en:
https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml

El siguiente trozo de código permite recuperar el archivo xml en cuestion, y lo almacena en disco.

In [ ]:
import urllib.request
x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml')
a=x.read()
a=a.decode("utf-8")
prueba=open("catalogo.xml","w")
prueba.write(str(a))
prueba.close()

__Ejercicio 1 [5 puntos]__

Se pide mostrar al usuario un listado de los nombres de las bibliotecas que contiene el archivo con el objetivo de que elija una de las bibliotecas y se le muestre de forma amigable(sin etiquetas) por pantalla la información asociada al parque que se encuentra en el fichero recuperado:

* Nombre de la biblioteca: atributo NOMBRE
* Horario: atributo HORARIO
* Equipamiento: atributo EQUIPAMIENTO
* Transporte: atributo TRANSPORTE
* Localización: atributos CLASE-VIAL,NOMBRE-VIA,NUM,LOCALIDAD,PROVINCIA,CODIGO-POSTAL 
* Teléfono: atributo TELÉFONO
* Email: atributo EMAIL  

Observar que algunos atributos pueden no aparecer en las bibliotecas. En estos casos, simplemente se mostrará vacío o con la palabra "Desconocido"

Por ejemplo si el usuario hubiera elegido la biblioteca "Biblioteca Pública Antonio Mingote (Latina)" debería mostrarse por pantalla:

__Nombre de la biblioteca:__

Biblioteca Pública Antonio Mingote (Latina)
    
__Horario:__

De lunes a viernes de 9 a 21 horas. Sala Infantil de 15 a 21 horas. Sábados de 9 a 14 horas. Los servicios de préstamo y acceso a internet finalizan 15 minutos antes del cierre de la Biblioteca.Días de cierre: Sábados: 11 de abril; 16 de mayo; todos los sábados de julio, agosto y septiembre; 26 de diciembre. Domingos: todos. Festivos:1 y 6 de enero; 9 y 10 de abril; 1, 2 y 15 de mayo; 15 de agosto; 12 de octubre; 2 y 9 de noviembre; 7, 8, 24, 25 y 31 de diciembre.

__Equipamiento:__

184 puestos de lectura y 25 para el uso de InternetWifi gratuito
    
__Transporte:__ 

Bus: 17, 138 Cercanías Renfe: Fanjul (línea C5)

__Localización:__ 

CALLE RAFAEL FINAT, 51.MADRID. MADRID 28044

__Teléfono:__ 

915 093 625

__Email:__ 

bib.latina@madrid.org

In [ ]:
from xml.etree import ElementTree

arbol = ElementTree.parse("201747-0-bibliobuses-bibliotecas.xml")

#bucle principal
while True:
    i=1
    print("0 : Salir")

    #iteramos por los nodos del árbol y vamos imprimiendo los nombres de las bibliotecas
    for nodo in arbol.findall("./contenido/atributos/atributo"):
        if(nodo.attrib.get("nombre") == "NOMBRE"):
            print(i, ":", nodo.text)
            i+=1
    print('\n')
    
    numBusqueda = input('\033[1m' + "Escribe el número de una biblioteca: ") #el caracter especial es para la negrita
    print('\n')
    if numBusqueda == "0": #si seleccinamos 0 salimos del bucle
        break

    i=1
    #iteramos por los nodos hasta encontrar nuestra biblioeca e imprimimos datos cuando
    #es alguno de los que nos interesa (nombre, horario, etc)
    for nodo in arbol.findall("./contenido/atributos"):
        if(str(i)==numBusqueda):
            for hijo in nodo.findall("./atributo"):
                found = True
                if(hijo.attrib.get("nombre") == "NOMBRE"):
                    print('\033[1m' + "Nombre de la biblioteca:")
                elif(hijo.attrib.get("nombre") == "HORARIO"):
                    print('\033[1m' + "Horario:")
                elif(hijo.attrib.get("nombre") == "EQUIPAMIENTO"):
                    print('\033[1m' + "Equipamiento:")
                elif(hijo.attrib.get("nombre") == "TRANSPORTE"):
                    print('\033[1m' + "Transporte:")
                elif(hijo.attrib.get("nombre") == "LOCALIZACION"): #imprimimos los nodos hijos de localizacion
                    print('\033[1m' + "Localización:" + '\033[0m\n')
                    indice = 1
                    found = False
                    for nieto in hijo.findall("./atributo"):
                        indice+=1
                        if indice>=9:
                            break
                        if(indice == 8):
                            print(nieto.text, '\n')
                        else:
                            print(nieto.text, end=', ')
                elif(hijo.attrib.get("nombre") == "DATOSCONTACTOS"): #igual con el contacto
                    found = False
                    for nieto in hijo.findall("./atributo"):
                        if(nieto.attrib.get("nombre") == "TELEFONO"):
                            print('\033[1m' + "Teléfono:" + '\033[0m\n')
                            print(nieto.text + '\n')
                        if(nieto.attrib.get("nombre") == "EMAIL"):
                            print('\033[1m' + "Email:" + '\033[0m\n')
                            print(nieto.text + '\n') 
                else:
                    found = False #si el nodo en el que estamos no corresponde con lo que queremos, no se imprime
                if(found):
                    print('\n' + '\033[0m' + hijo.text + '\n')
            break
        else:
            i+=1
    if(i>52): #si nuestro indice no es 0 y no corresponde a ninguna biblioteca de la lista
        print("El número está fuera de rango.\n")

__Ejercicio 2 [5 puntos]__

Este ejercicio se pide crear un buscador sobre la información recuperada, de forma que el usuario podrá buscar bibliotecas por diferentes criterios:

 * Accesibilidad
 
 * Nombre
 
 * Barrio
 
 * Distrito
 
Como resultado debería mostrarse un listado de todas las bibliotecas que cumplen las condiciones impuestas por el usuario. Se mostrará de cada biblioteca la misma información que en el caso anterior.  En caso de no existir una biblioteca con las condiciones dada, se mostrará un mensaje informativo.

Observar que habrá que preguntar al usuario por cada uno de los criterios, y éste tendrá que seleccionar un valor por cada criterio o no seleccionarlo. Después de mostrarle la información, se le volverá a preguntar si quiere seguir buscando. Entre las opciones del menú deberá existir una que sea para finalizar la búsqueda.

In [ ]:
from xml.etree import ElementTree
#convertimos el archivo en un element tree
arbol=ElementTree.parse("201747-0-bibliobuses-bibliotecas.xml")

#funcion que dado un diccionario con los filtros que queremos aplicar escribe la 
#información de las bibliotecas que cumplan dichos requisitos o en caso
#de que ninguna lo cumpla muestra un mensaje de error
def busca(a):
    raiz=arbol.getroot()
    hayBiblioteca = False;
    for nodo in raiz.findall("contenido"):
        cumple = compruebaBiblioteca(nodo, a)
        if(cumple): 
            muestraBiblioteca(nodo)
            hayBiblioteca = True
            
    if (not hayBiblioteca): 
        print("No se ha encontrado ninguna Biblioteca con los filtros añadidos")   

#funcion implementada en el apartado anterior para mostrar la informacion de una biblioteca
def muestraBiblioteca(nodo):
    for hijo in nodo.findall("atributos/atributo"):
        found = True
        if(hijo.attrib.get("nombre") == "NOMBRE"):
            print('\033[1m' + "Nombre de la biblioteca:")
        elif(hijo.attrib.get("nombre") == "HORARIO"):
            print('\033[1m' + "Horario:")
        elif(hijo.attrib.get("nombre") == "EQUIPAMIENTO"):
            print('\033[1m' + "Equipamiento:")
        elif(hijo.attrib.get("nombre") == "TRANSPORTE"):
            print('\033[1m' + "Transporte:")
        elif(hijo.attrib.get("nombre") == "LOCALIZACION"):
            print('\033[1m' + "Localización:" + '\033[0m\n')
            indice = 1
            found = False
            for nieto in hijo.findall("./atributo"):
                indice+=1
                if indice>=9:
                    break
                if(indice == 8):
                    print(nieto.text, '\n')
                else:
                    print(nieto.text, end=', ')
        elif(hijo.attrib.get("nombre") == "DATOSCONTACTOS"):
            found = False
            for nieto in hijo.findall("./atributo"):
                if(nieto.attrib.get("nombre") == "TELEFONO"):
                    print('\033[1m' + "Teléfono:" + '\033[0m\n')
                    print(nieto.text + '\n')
                if(nieto.attrib.get("nombre") == "EMAIL"):
                    print('\033[1m' + "Email:" + '\033[0m\n')
                    print(nieto.text + '\n') 
        elif(hijo.attrib.get("nombre") == "TRANSPORTE"):
            print('\033[1m' + "Transporte:")
        else:
            found = False
        if(found):
            print('\n' + '\033[0m' + hijo.text + '\n')

#funcion que recorre los atributos de la biblioteca y comprueba que dicha biblioteca cumpla los 
#criterios definidos anteriormente 
def compruebaBiblioteca(nodo, a):
    correcto = True;
    for atributos in nodo.findall("atributos/atributo"):
            
        if(a["accesibilidad"] != "" and atributos.attrib.get("nombre") == "ACCESIBILIDAD" 
            and a["accesibilidad"] not in atributos.text):
            correcto = False
            break
        elif(a["nombre"] != "" and atributos.attrib.get("nombre") == "NOMBRE" 
            and atributos.text != a["nombre"].upper()):
            correcto = False
            break
        elif atributos.attrib.get("nombre") == "LOCALIZACION":
             for localizacion in atributos.findall("atributo"):
                    if(a["barrio"] != "" and localizacion.attrib.get("nombre") == "BARRIO" 
                       and localizacion.text != a["barrio"].upper()):
                        correcto = False
                        break
                    elif(a["distrito"] != "" and localizacion.attrib.get("nombre") == "DISTRITO" 
                       and localizacion.text != a["distrito"].upper()):
                        correcto = False
                        break
            
    
    return correcto
    
#función que gestiona la introduccion de filtros, la salida del programa y la búsqueda de
#biblioteca segun los filtros previamente mencionados
def printMenu():
    filtros = {"accesibilidad": "", "nombre": "", "barrio": "", "distrito": ""} 
    modificador = "-1"
    while modificador != "5":
        modificador = input("Seleccione para añadir un filtro: \n\t 0:Accesibilidad, \n\t 1:Nombre, \n\t 2:Barrio, \n\t 3:distrito" +
                           "\n Pulse 4 para realizar la búsqueda con los filtros actuales." +
                           "\n Pulse 5 para salir del programa.\n")
        
        if(modificador == "0"):
            entrada = input("Seleccione criterio para filtrar bibliotecas segun su Accesibilidad: ")
            filtros["accesibilidad"] = entrada
        elif(modificador == "1"):
            entrada = input("Seleccione criterio para filtrar bibliotecas segun su nombre: ")
            filtros["nombre"] = entrada
        elif(modificador == "2"):
            entrada = input("Seleccione criterio para filtrar bibliotecas segun su barrio: ")
            filtros["barrio"] = entrada
        elif(modificador == "3"):
            entrada = input("Seleccione criterio para filtrar bibliotecas segun su distrito: ")
            filtros["distrito"] = entrada
        elif(modificador == "4"): busca(filtros)
            
            
            

printMenu()